# Generate Training Data

**Author:** Manuela Linke & Gabriel Micard, HTWG Konstanz 

**Date:** 14.04.2024 

**Summary:** This Script generates the training data for the training of the CNN. For more efficient generation through the application of parallelisation please use the Generate_Training_data.py Script as described in the README-File.

Three steps:
1) generation of load and generation data (all loads/generations vary within the values Pset, Pmax, 3*Pmax, - Pmax (to simulate PV generation))
2) identification of problematic grid states (in basic switch and transformer configuration)
3) finding the optimal solution for each problematic grid state individually
    - best switching configuration with minimum change of topology
    - if no configuration is able to bring the network into the specifications, it is indicated to the user and the configuration leading to the minimium deviation to the specification is indicated

![Flowchart](../doc/img/Dataset_generation.png)

In [1]:
# Load python packages
from tqdm import tqdm
import numpy as np
import scipy 
import math
import time as t
from tqdm import tqdm
import datetime as dt
import os
import itertools as it
import pandas as pd

import pandapower as pp
from pandapower.plotting import simple_plot, pf_res_plotly
from pandapower.plotting.plotly import simple_plotly
import matplotlib.pyplot as plt

## Import grid 
Saved in standart p format, load the set and maximum value for each load

Our example grid: *Cossmic grid*

In [2]:
import os

#import the grid 
net = pp.from_pickle("cossmic_grid.p")

#import the loads file separately to get information about p_max and p_set values
loads = pd.read_table(os.path.join('..', '0_data preparation', 'Cossmic_grid', 'grid-data', 'loads.csv'), sep=",")

#import the allowed switch configurations
allowed_sw = np.load("allowed_switches.npy") 

# define the base configuration for the switches and transformer tap positions
base_switch = net.switch.closed.loc[net.switch.et == 'l']        
base_trafo = net.trafo.tap_pos  

# set seed for generating random data
seed = 42

In [3]:
# define filenames with timestamp
ts = t.time()
strTime = dt.datetime.fromtimestamp(ts).strftime('%Y-%m-%d_%H-%M-%S')
folder = os.path.join('data', 'preprocessed')
if not os.path.exists(folder):
    os.makedirs(folder)

## Ansatz 1: Generate all combinations 
In case of the example grid 4.194.304 combinations

In [4]:
state_lists = []
for _, row in loads.iterrows():
    states = [row['p_max'], row['p_set'], -row['p_max'], row['p_max']*3]
    state_lists.append(states)

# generate all possible combinations
all_combinations = list(it.product(*state_lists))

In [5]:
#result_df = pd.DataFrame(all_combinations, columns=loads.name)

## Ansatz 2: Nur 3 x p_max oder 3 x -p_max
- 3 loads are set at 3*pmax or at -pmax (to simulate PV generation)
- the other loads are set at pmax or pset 

In [6]:
state_lists = []
for _, row in loads.iterrows():
    states = [row['p_max'], row['p_set']]
    state_lists.append(states)

# generate all possible combinations (in case of Cossmic Grids: 2048)
all_combinations = list(it.product(*state_lists))

df_combinations = pd.DataFrame(all_combinations, columns=loads.name)

In [7]:
# setting all loads to false
original_list = [False] * len(loads.name)

# generate combinations of 3 
index_combinations = list(it.combinations(range(len(loads.name)), 3))

modified_lists = []

# Modify the list by the generated combinations
for indices in index_combinations:

    modified_list = original_list.copy()
    
    # Setting the values to true of the combinations
    for index in indices:
        modified_list[index] = True
    
    # append to list
    modified_lists.append(modified_list)

In [8]:
t0 = t.time()
result_list = []

for comb_index in df_combinations.index:
    this_comb = df_combinations.loc[comb_index].copy()  
    #print(this_comb)

    for modified_list in modified_lists:
        modified_comb_1 = this_comb.copy()  # Copy the current combination for modifications
        modified_comb_2 = this_comb.copy()  # Copy the current combination for modifications
        for index, element in enumerate(modified_list):
            if element: 

                # Replace with the corresponding value from loads.p_max*3 or -loads.p_max
                modified_comb_1.iloc[index] = loads.p_max.iloc[index] * 3
                modified_comb_2.iloc[index] = -loads.p_max.iloc[index]
        result_list.append(modified_comb_1)
        result_list.append(modified_comb_2)

all_results_df = pd.DataFrame(result_list, columns=loads.name)
all_results_df.reset_index(drop=True, inplace=True)
all_results_df = all_results_df.sample(frac=1, random_state=seed).reset_index(drop=True)
ordered_columns = net.load['name'].tolist()
all_results_df = all_results_df[ordered_columns]
t1 = t.time()
t_elapsed = t1-t0
print("Time elapsed: %.1f sec." %t_elapsed)

Time elapsed: 196.8 sec.


In [9]:
len(all_results_df)

675840

In [10]:
# save generated load data
all_results_df.to_feather("data/raw/" + strTime + "_time_series_data.feather")
#all_results_df = pd.read_feather(file_path)

In [10]:
# Since all_results_df has 675840 entries, take a smaller part of it:
result_df = all_results_df[:1000]

In [11]:
result_df.head()

name,ind_1,ind_2,ind_3,res_1,res_2,res_3,res_4,res_5,res_6,pub_1,pub_2
0,0.033,0.0112,0.2560,0.0067,0.00810,0.00520,0.00054,-0.00370,-0.00350,-0.0200,0.1920
1,0.163,-0.0112,-0.2560,0.0067,0.00028,-0.00520,0.00054,0.00370,0.00350,0.0200,0.0596
2,0.163,0.0016,0.2560,0.0067,0.00028,0.00520,0.00054,-0.00370,-0.00350,-0.0200,0.1920
3,0.163,0.0112,0.7680,0.0067,0.00028,0.01560,0.01740,0.00032,0.00036,0.0027,0.1920
4,0.033,0.0112,0.0797,-0.0067,0.00028,0.00044,0.00054,-0.00370,0.00350,0.0200,-0.1920


### Identify the problematic situations 
- simulate each previously defined situation with the basic switch configuration and trafo configuration  

In [12]:
t0 = t.time()

## define the storage variables  
Voltage_mat = np.empty(shape=[0, len(net.bus[net.bus['name'].isin(net.load['name'])])])
Load_mat = np.empty(shape=[0, len(net.load.p_mw)])
Problem_num = 0
Problem_detail_mat = np.empty(shape = [0, len(net.bus) + len(net.trafo) + len(net.line)])

notconverged=0           ## define a counter for not converged simulations

ordered_columns = net.load['name'].tolist()
result_df = result_df[ordered_columns]

for timestamp, row in result_df.iterrows():
    net.load['p_mw'] = row.values

    try:
        ## run the power flow simulation 
        pp.runpp(net, numba = True) #, numba=False)                                        

        Trafo_overload = np.asarray(net.res_trafo.loading_percent>100)         
        Bus_voltage_offspec = np.asarray((net.res_bus.vm_pu>1.03)|(net.res_bus.vm_pu<0.97)) 
        Line_overload = np.asarray(net.res_line.loading_percent>100) 
        PB_V = np.concatenate((Trafo_overload, Bus_voltage_offspec, Line_overload))

        Problem = np.sum(PB_V) > 0 
        
        if Problem:                                               
            Voltage_mat = np.append(Voltage_mat, [net.res_bus.vm_pu[net.bus['name'].isin(net.load['name'])]], axis=0)     
            Load_mat = np.append(Load_mat, [net.load.p_mw.values], axis=0)                                           
            Problem_detail_mat=np.append(Problem_detail_mat, PB_V) 
            Problem_num += 1 
    except:
        ## if not converged then increment the counter
        notconverged += 1           

t1 = t.time()
execution_time_minutes = (t1-t0)/60
print(f"Total execution time: {execution_time_minutes:.2f} minutes")
print("There are", Problem_num, "problematic cases out of", len(result_df))
print("There are ", notconverged, "cases in which the simulation did not converge")

Total execution time: 0.48 minutes
There are 436 problematic cases out of 1000
There are  0 cases in which the simulation did not converge


### Save and test

In [13]:
# save the data
np.save(folder + "/" + strTime + "_" + "voltage_distribution.npy", Voltage_mat)
np.save(folder + "/" + strTime + "_" + "Load_distribution.npy", Load_mat)
np.save(folder + "/" + strTime + "_" + "Problem_details.npy",Problem_detail_mat)

### Look for a solution
for every problematic case found in the previous step
- test all allowed switch configurations, and allowed trafo configuration 

Determine if there is a solution that satisfies the criteria

- minimal change from base configuration
- minimal deviation from the nominal voltage
 
If no solution is found the configuration that minimizes the voltage deviation is used.

The output training vector for the KNN has 15 variables
[ switch 1->10 (0,1) ,trafo 1 & 2 (-2, -1, 0, 1, 2), solution or not (0,1)] 

In [ ]:
t0 = t.time()

Sol_list=[]

# Define output counters
impossible_c = 0
possible_c = 0
notconverged=0 

# Define loop variables
tap_pos_ranges = [range(row['tap_min'], row['tap_max'] + 1) for index, row in net.trafo.iterrows()]
op_length = len(net.switch[net.switch.et == 'l']) + len(net.trafo) + 1 #add 1 dimension for indicating if there is a solution or not
max_changes = len(net.switch[net.switch.et == 'l']) + sum(abs(net.trafo.tap_min - net.trafo.tap_max))

# initializing arrays for storing the solutions
min_chg_from_base_vector=np.empty(shape=[0, op_length],dtype=int)            
min_dev_from_norm_V_vector=np.empty(shape=[0, op_length],dtype=int)

for d in tqdm(range(len(Load_mat))) :
    
    #initializing minimal values and temporal solution arrays
    min_dev_from_norm_V = np.inf
    min_dev_from_norm_V_conf = np.empty(shape=[op_length],dtype=int)
    min_dev_from_norm_pb_V = np.inf
    min_dev_from_norm_pb_V_conf = np.empty(shape=[op_length],dtype=int)
    min_chg_from_base = max_changes
    min_chg_from_base_conf = np.empty(shape=[op_length],dtype=int)

    #initializing counters
    no_pb_cnt = 0
    notconverged = 0
    
    net.load['p_mw'] = Load_mat[d]
       
    for switches in range(len(allowed_sw)):         
        net.switch.loc[net.switch.et == 'l', 'closed'] = allowed_sw[switches]       
        
        supplied = len(pp.topology.unsupplied_buses(net)) == 0
        if (supplied == False):
            print("The following buses are unsupplied:", pp.topology.unsupplied_buses(net))
            
        for combination in it.product(*tap_pos_ranges):
            net.trafo.tap_pos = combination
            
            try: 
                pp.runpp(net, numba = True)            

                Trafo_overload = np.asarray(net.res_trafo.loading_percent>100)        
                Bus_voltage_offspec = np.asarray((net.res_bus.vm_pu>1.03)|(net.res_bus.vm_pu<0.97))  
                Line_overload = np.asarray(net.res_line.loading_percent>100) 
                
                PB_V = np.concatenate((Trafo_overload, Bus_voltage_offspec, Line_overload))                                      
                no_problem = np.sum(PB_V) == 0      

                ## sum the square of the deviation of the pu-voltage for all buses
                dev_from_norm_V = np.sum((net.res_bus.vm_pu-1)**2) 
                chg_from_base = np.sum(base_switch ^ allowed_sw[switches]) + np.sum(base_trafo ^ net.trafo.tap_pos.values)

                # in case no solution will be found
                if (dev_from_norm_V < min_dev_from_norm_pb_V):
                    min_dev_from_norm_pb_V = dev_from_norm_V
                    min_dev_from_norm_pb_V_conf = np.append(np.append(allowed_sw[switches]*1, net.trafo.tap_pos.values.tolist()), 0)
                    
                if no_problem :                                 
                    no_pb_cnt +=1  
                        
                    if (dev_from_norm_V < min_dev_from_norm_V):
                        min_dev_from_norm_V = dev_from_norm_V
                        min_dev_from_norm_V_conf = np.append(np.append(allowed_sw[switches]*1, net.trafo.tap_pos.values.tolist()), 1)
                        
                        ## sum all the changes to the base configuration 
                    if (chg_from_base < min_chg_from_base):  
                        min_chg_from_base = chg_from_base      
                        ## this configuraion is stored as the minmum for this criteria
                        min_chg_from_base_conf = np.append(np.append(allowed_sw[switches]*1, net.trafo.tap_pos.values.tolist()), 1)
                      
            except:  
                notconverged += 1 #increase the non convergence counter                       

    # if a solution was found
    if (no_pb_cnt > 0):                
        possible_c += 1
        
        min_chg_from_base_vector = np.append(min_chg_from_base_vector, [min_chg_from_base_conf], axis=0)  #save the confs in matrices
        min_dev_from_norm_V_vector = np.append(min_dev_from_norm_V_vector, [min_dev_from_norm_V_conf], axis=0)

    #if there is no sultion
    else:
        #print("For Case", d, "there is no solution, therefore minimizing the deviation of the nominal voltage")  
        impossible_c += 1
        #save the conf that minimize the V deviation for every criteria 
        min_chg_from_base_vector = np.append(min_chg_from_base_vector, [min_dev_from_norm_pb_V_conf], axis=0)
        min_dev_from_norm_V_vector = np.append(min_dev_from_norm_V_vector, [min_dev_from_norm_pb_V_conf], axis=0) 
        

print("-------------------------------------------------------------------------------")
print("We analyzed", len(Load_mat), "problematic load configurations:")
print("There are", possible_c, "possible cases with a solution ", 100*possible_c/(d+1),"%")
print("There are", impossible_c, "impossible cases (without a solution) ", 100*impossible_c/(d+1),"%")
print("Number of possible switching states:", len(allowed_sw)*math.prod(abs(net.trafo.tap_min - net.trafo.tap_max) + 1))            
##when every calculation is finished 
t1 = t.time()
execution_time_minutes = (t1-t0)/60
print(f"Total execution time: {execution_time_minutes:.2f} minutes")

  1%|▉                                                                              | 5/436 [07:59<11:22:01, 94.94s/it]

In [ ]:
filename_chg = "_min_chg_from_base.npy"
filename_dev = "_min_dev_from_norm_V.npy"

np.save(folder + "/" + strTime + "_" + filename_dev, min_dev_from_norm_V_vector)
np.save(folder + "/" + strTime + "_" + filename_chg, min_chg_from_base_vector)

## Testing the results at example of Load_mat[0]

In [ ]:
from pandapower.plotting.plotly import pf_res_plotly
d=0
net.switch.loc[net.switch.et == 'l', 'closed'] = base_switch      
net.trafo.tap_pos = base_trafo
print("Load configuration :", Load_mat[d])
net.load["p_mw"] = Load_mat[d]
pp.runpp(net) 
print("-------------")
print("Trafo loading :", net.res_trafo.loading_percent)
print("-------------")
print("Node voltage :", net.res_bus.vm_pu)
print("-------------")
print("Line loading :", net.res_line.loading_percent)
pf_res_plotly(net)

In [ ]:
# Defining start and end index of the switches and transformers
start_index = len(net.switch[net.switch.et == 'l'])
end_index = start_index + len(net.trafo)

# Extracting Transformer station combinations
trafo_comb = min_dev_from_norm_V_vector[start_index:end_index]

sw_comb = min_dev_from_norm_V_vector[d, :start_index].astype(bool)
# Extracting Transformer station combinations
trafo_comb = min_dev_from_norm_V_vector[d, start_index:end_index]
solvable = min_dev_from_norm_V_vector[d, end_index:]

print("Is this problem solvable?: ", solvable.astype(bool))
print("Changing the switches to: ", sw_comb)
print("Changing the Transformer tap positions to:", trafo_comb)

net.switch.loc[net.switch.et == 'l', 'closed'] = sw_comb       
net.trafo.tap_pos = trafo_comb

In [ ]:
min_dev_from_norm_V_vector[:,end_index:]

unique_elements, counts = np.unique(min_dev_from_norm_V_vector[:,end_index:], return_counts=True)

# Print results
for element, count in zip(unique_elements, counts):
    print(f"{element} occurs {count} times.")

In [ ]:
pp.runpp(net) 
print("-------------")
print("Trafo loading :", net.res_trafo.loading_percent)
print("-------------")
print("Node voltage :", net.res_bus.vm_pu)
print("-------------")
print("Line loading :", net.res_line.loading_percent)

In [ ]:
pf_res_plotly(net)